In [ ]:
#extraire toutes les dépendants des lemmes ADJ et ADV avec des déterminants possessifs

In [10]:
short_ep1 = "/home/port-pret-etu01/Documents/LATTICE/pivotsents/short/pivots_NEW1810-1840_pain_short.txt"
short_ep2 = "/home/port-pret-etu01/Documents/LATTICE/pivotsents/short/pivots_NEW1841-1913_pain_short.txt"
short_ep3 = "/home/port-pret-etu01/Documents/LATTICE/pivotsents/short/pivots_NEW1914-2009_pain_short.txt"

fichiers = [short_ep1, short_ep2, short_ep3]

In [11]:
import os
import json
import pandas as pd
from collections import Counter
from tabulate import tabulate

pronoms = {"his", "her", "my", "their", "your", "our", "thy"}

def lire_tokens(chemin):
    with open(chemin, "r", encoding="utf-8") as f:
        lignes = [ligne.strip() for ligne in f if ligne.strip()]
    tokens = []
    for ligne in lignes:
        tokens.extend(ligne.split())
    return tokens

def compter_pronoms_avant(tokens, mot_cible):
    compte_pronoms = Counter()
    total_pronom = 0
    total_mot = 0
    for i in range(len(tokens)):
        if tokens[i].lower() == mot_cible:
            total_mot += 1
            if i > 0:
                precedent = tokens[i - 1].lower()
                if precedent in pronoms:
                    compte_pronoms[precedent] += 1
                    total_pronom += 1
    return total_mot, total_pronom, compte_pronoms

def detect_epoque(path):
    if "1810-1840" in path:
        return "ep1"
    elif "1841-1913" in path:
        return "ep2"
    elif "1914-2009" in path:
        return "ep3"

with open("/home/port-pret-etu01/Documents/LATTICE/méta/output/total_pivot_counts.json", "r", encoding="utf-8") as f:
    pivot_totals = json.load(f)

fichier_sortie = "/home/port-pret-etu01/Documents/LATTICE/pivotsents/short/pronouns"

with open(fichier_sortie, "w", encoding="utf-8") as out:
    for chemin in fichiers:
        tokens = lire_tokens(chemin)
        ep = detect_epoque(chemin)

        painful_total = pivot_totals.get(ep, {}).get("painful", 0)
        painfully_total = pivot_totals.get(ep, {}).get("painfully", 0)

        total_painful, total_painful_pronom, compte_painful = compter_pronoms_avant(tokens, "painful")
        total_painfully, total_painfully_pronom, compte_painfully = compter_pronoms_avant(tokens, "painfully")

        out.write(f"Fichier : {chemin}\n\n")

        out.write(f"painful - Nombre total (pivot JSON) : {painful_total}\n")
        out.write(f"painful - Nombre précédé d'un pronom : {total_painful_pronom}\n\n")
        if total_painful_pronom > 0:
            df_painful = pd.DataFrame(compte_painful.most_common(), columns=["Pronom", "Fréquence"])
            df_painful["Part / TOTAL (%)"] = df_painful["Fréquence"].apply(
                lambda x: round((x / painful_total) * 100, 2) if painful_total > 0 else 0
            )
            tableau_painful = tabulate(df_painful, headers="keys", tablefmt="fancy_grid", showindex=False)
            out.write(tableau_painful + "\n\n")
        else:
            out.write("rien trouvé".\n\n")

        out.write(f"painfully - Nombre total (pivot JSON) : {painfully_total}\n")
        out.write(f"painfully - Nombre précédé d'un pronom : {total_painfully_pronom}\n\n")
        if total_painfully_pronom > 0:
            df_painfully = pd.DataFrame(compte_painfully.most_common(), columns=["Pronom", "Fréquence"])
            df_painfully["Part / TOTAL (%)"] = df_painfully["Fréquence"].apply(
                lambda x: round((x / painfully_total) * 100, 2) if painfully_total > 0 else 0
            )
            tableau_painfully = tabulate(df_painfully, headers="keys", tablefmt="fancy_grid", showindex=False)
            out.write(tableau_painfully + "\n\n")
        else:
            out.write("rien trouvé.\n\n")

        out.write("=" * 80 + "\n\n")

print(f"Saved under '{fichier_sortie}'")


Résultats sauvegardés dans '/home/port-pret-etu01/Documents/LATTICE/pivotsents/short/pronouns'
